In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import utils
from nets.NetOneLayer import NetOneLayer
from nets.NetOneLayerLowRank import NetOneLayerLowRank

In [3]:
batch_size = 128
batch_size_test = 1000

train_loader, test_loader = utils.load_mnist(batch_size, batch_size_test)

Processing...
Done!


In [4]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
lr = 0.02
momentum = 0.9
n_epochs = 100

model = NetOneLayerLowRank(n_hidden=2**8, d=2, K=2)
# model = NetOneLayer(n_hidden=2**8)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(n_epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.400447
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.266492
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.140635

Test set: Average loss: -0.2874, Accuracy: 2948/10000 (29%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.145363
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.123567
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.167380

Test set: Average loss: -0.3667, Accuracy: 3715/10000 (37%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.146219
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.033773
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.054403

Test set: Average loss: -0.4059, Accuracy: 4120/10000 (41%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.051655
Train Epoch: 3 [25600/60000 (43%)]	Loss: 2.000755
Train Epoch: 3 [51200/60000 (85%)]	Loss: 1.980679

Test set: Average loss: -0.4335, Accuracy: 4372/10000 (44%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.906554
Train Epoch: 4 [25600/60000 (43%)]	Loss: 2.004432
Train Epoch: 4 [51200/60000 (85%)]	Loss: 2.048994

Test set: Average loss: -0

Train Epoch: 39 [25600/60000 (43%)]	Loss: 1.824837
Train Epoch: 39 [51200/60000 (85%)]	Loss: 1.806263

Test set: Average loss: -0.6273, Accuracy: 6305/10000 (63%)

Train Epoch: 40 [0/60000 (0%)]	Loss: 1.855736
Train Epoch: 40 [25600/60000 (43%)]	Loss: 1.879616
Train Epoch: 40 [51200/60000 (85%)]	Loss: 1.851143

Test set: Average loss: -0.6275, Accuracy: 6309/10000 (63%)

Train Epoch: 41 [0/60000 (0%)]	Loss: 1.818695
Train Epoch: 41 [25600/60000 (43%)]	Loss: 1.807518
Train Epoch: 41 [51200/60000 (85%)]	Loss: 1.832997

Test set: Average loss: -0.6285, Accuracy: 6318/10000 (63%)

Train Epoch: 42 [0/60000 (0%)]	Loss: 1.855214
Train Epoch: 42 [25600/60000 (43%)]	Loss: 1.770466
Train Epoch: 42 [51200/60000 (85%)]	Loss: 1.825451

Test set: Average loss: -0.6296, Accuracy: 6323/10000 (63%)

Train Epoch: 43 [0/60000 (0%)]	Loss: 1.825956
Train Epoch: 43 [25600/60000 (43%)]	Loss: 1.810304
Train Epoch: 43 [51200/60000 (85%)]	Loss: 1.852389

Test set: Average loss: -0.6294, Accuracy: 6326/10000 (63

Train Epoch: 78 [51200/60000 (85%)]	Loss: 1.813203

Test set: Average loss: -0.7032, Accuracy: 7057/10000 (71%)

Train Epoch: 79 [0/60000 (0%)]	Loss: 1.728804
Train Epoch: 79 [25600/60000 (43%)]	Loss: 1.705717
Train Epoch: 79 [51200/60000 (85%)]	Loss: 1.697637

Test set: Average loss: -0.7047, Accuracy: 7071/10000 (71%)

Train Epoch: 80 [0/60000 (0%)]	Loss: 1.690799
Train Epoch: 80 [25600/60000 (43%)]	Loss: 1.834146
Train Epoch: 80 [51200/60000 (85%)]	Loss: 1.752124

Test set: Average loss: -0.7050, Accuracy: 7074/10000 (71%)

Train Epoch: 81 [0/60000 (0%)]	Loss: 1.726684
Train Epoch: 81 [25600/60000 (43%)]	Loss: 1.772324
Train Epoch: 81 [51200/60000 (85%)]	Loss: 1.702155

Test set: Average loss: -0.7052, Accuracy: 7075/10000 (71%)

Train Epoch: 82 [0/60000 (0%)]	Loss: 1.677474
Train Epoch: 82 [25600/60000 (43%)]	Loss: 1.745079
Train Epoch: 82 [51200/60000 (85%)]	Loss: 1.759999

Test set: Average loss: -0.7051, Accuracy: 7066/10000 (71%)

Train Epoch: 83 [0/60000 (0%)]	Loss: 1.757871
T